In [ ]:
#Final version
import cv2
import matplotlib.pyplot as plt

    # Path to video file
vidObj = cv2.VideoCapture("replace with your video path")

count = 0

success = 1

while success:

        # vidObj object calls read
        # function extract frames
  success, image = vidObj.read()

  outdir = "replace with your desired folder to ouput to"

  os.makedirs(outdir, exist_ok=True)
  # Names each frame
  filepath = os.path.join(outdir, f"{count}.jpg")
  cv2.imwrite(filepath, image)


    # Saves the frames with frame-count

  count += 1



In [ ]:
import pygame
import numpy as np
import os
from scipy.ndimage import gaussian_filter
from scipy.spatial import Delaunay
from collections import defaultdict
import matplotlib.pyplot as plt
import pygame.gfxdraw

def final(image):
    inp = pygame.surfarray.pixels3d(pygame.image.load(image))
    perceptual_weight = np.array([0.2126, 0.7152, 0.0722])
    grayscale = (inp * perceptual_weight).sum(axis=-1)
    x = gaussian_filter(grayscale, 2, mode="reflect")
    x2 = gaussian_filter(grayscale, 30, mode="reflect")

    diff = (x - x2)
    diff[diff < 0] *= 0.1
    diff = np.sqrt(np.abs(diff) / diff.max())

    def sample(ref, n=1000000):
        np.random.seed(0)
        w, h = x.shape
        xs = np.random.randint(0, w, size=n)
        ys = np.random.randint(0, h, size=n)
        value = ref[xs, ys]
        accept = np.random.random(size=n) < value
        points = np.array([xs[accept], ys[accept]])
        return points.T, value[accept]

    samples, v = sample(diff)
    plt.scatter(samples[:, 0], -samples[:, 1], c=v, s=0.2, edgecolors="none", cmap="viridis")

    def get_colour_of_tri(tri, image):
        colours = defaultdict(lambda: [])
        w, h, _ = image.shape
        for i in range(0, w):
            for j in range(0, h):
                index = tri.find_simplex((i, j))
                colours[int(index)].append(inp[i, j, :])
        for index, array in colours.items():
            colours[index] = np.array(array).mean(axis=0)
        return colours

    def draw(tri, colours, screen, upscale):
        s = screen.copy()
        for key, c in colours.items():
            t = tri.points[tri.simplices[key]]
            pygame.gfxdraw.filled_polygon(s, t * upscale, c)
            pygame.gfxdraw.polygon(s, t * upscale, c)
        return s

    w, h, _ = inp.shape
    upscale = 2
    screen = pygame.Surface((w * upscale, h * upscale))
    screen.fill(inp.mean(axis=(0, 1)))
    corners = np.array([(0, 0), (0, h - 1), (w - 1, 0), (w - 1, h - 1)])
    points = np.concatenate((corners, samples))

    outdir = "replace with the desired folder to output to"
    os.makedirs(outdir, exist_ok=True)
    # Higher i value = Clearer image
    i=40
    n = 5 + i + 2 * int(i**2)
    tri = Delaunay(points[:n, :])
    colours = get_colour_of_tri(tri, inp)
    s = draw(tri, colours, screen, upscale)
    s = pygame.transform.smoothscale(s, (w, h))
    pygame.image.save(s, f"(replace with the desired folder to output to)/{count}.png")

In [ ]:
count=0;

for i in range (232):
  count=i
  final(f"(replace with the desired folder to take the frames from)/{i}.jpg")
  print(i)


In [ ]:
import os

frameArr = []
folder_path = '(replace with the desired folder to take the new frames from)'
contents = os.listdir(folder_path)

# Since the files are not in order, the line below helps to organise them
contents.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

for paths in contents:
    frameArr.append(folder_path + paths)
    print(frameArr[-1])  # Prints the most recently added path in frameArr

In [ ]:
import cv2
import os
from google.colab import files



if len(frameArr) == 0:
    print("No images found!")
else:
    img = cv2.imread(frameArr[0])
    height, width, layers = img.shape
    size = (width, height)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 25  # Frames per second
    out = cv2.VideoWriter('/content/videos_output.mp4', fourcc, fps, size)

    for image_path in frameArr:
        img = cv2.imread(image_path)
        out.write(img) # Writes each frame

    out.release()
    print("Video saved as '/content/videos_output.mp4'")
    files.download('/content/videos_output.mp4')
